# Modulos

In [ ]:
# Librería para manipulación y análisis de datos mediante DataFrames
import pandas as pd
import re
# Librería para manipulación de rutas de archivos y directorios
from pathlib import Path
# Librería para trabajar con fechas y horas
from datetime import datetime, timedelta

# Rutas y cargue de datafarmes

In [ ]:
r_ms_sie = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\SIE\Aseguramiento\ms_sie\Reporte_Validación Archivos Maestro_2025_12_01.csv"
R_Municipios_SIE = r"C:\Users\osmarrincon\OneDrive - 891856000_CAPRESOCA E P S\Capresoca\AlmostClear\SIE\codificación de variables categóricas\Reporte_MUNICIPIOS_2025_05_14.csv"
df_ms_sie = pd.read_csv(r_ms_sie, sep=';', dtype=str, encoding='latin-1')
df_Municipios_SIE = pd.read_csv(R_Municipios_SIE, sep=';', dtype=str, encoding='ANSI')

# Limpiesa de datos

In [ ]:
# Renombramos la columna "municipio" de df_Municipios_SIE a "ID_COD_municipio"
df_munis = df_Municipios_SIE.rename(columns={"municipio": "ID_COD_municipio"})[["descripcion", "ID_COD_municipio"]]

# Hacemos merge entre df_ms_sie y df_munis usando df_ms_sie["municipio"] y df_munis["descripcion"]
df_ms_sie = df_ms_sie.merge(df_munis, left_on="municipio", right_on="descripcion", how="left")

# Eliminamos la columna "descripcion" que ya no es necesaria
df_ms_sie.drop("descripcion", axis=1, inplace=True)

# Reordenamos las columnas para que "ID_COD_municipio" quede justo a la derecha de "municipio"
cols = list(df_ms_sie.columns)
idx = cols.index("municipio")
cols.remove("ID_COD_municipio")
cols.insert(idx + 1, "ID_COD_municipio")
Df_SIE = df_ms_sie[cols]

In [ ]:
df_ms_sie = df_ms_sie[['tipo_documento', 'numero_identificacion', 'fecha_nacimiento', 'genero', 'municipio', 'estado', 'regimen', 'direccion', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico', 'ID_COD_municipio']]

## Activos SIE

In [ ]:
# ANTES del filtro - Guardar la cantidad inicial
registros_iniciales = len(df_ms_sie)

print("=" * 60)
print("ESTADO ANTES DEL FILTRO")
print("=" * 60)
print(f"\nTotal de registros: {registros_iniciales}")
print(f"Número de columnas: {len(df_ms_sie.columns)}")
print(f"\nDistribución por estado:")
print(df_ms_sie['estado'].value_counts())
print(f"\nPorcentaje por estado:")
print(df_ms_sie['estado'].value_counts(normalize=True).round(4) * 100)

# APLICAR el filtro
df_ms_sie = df_ms_sie[df_ms_sie['estado'] == 'Activo']

# DESPUÉS del filtro - Calcular registros finales
registros_finales = len(df_ms_sie)
registros_eliminados = registros_iniciales - registros_finales
porcentaje_eliminado = (registros_eliminados / registros_iniciales * 100) if registros_iniciales > 0 else 0

print("\n" + "=" * 60)
print("ESTADO DESPUÉS DEL FILTRO")
print("=" * 60)
print(f"\nTotal de registros: {registros_finales}")
print(f"Número de columnas: {len(df_ms_sie.columns)}")
print(f"\nDistribución por estado:")
print(df_ms_sie['estado'].value_counts())

print("\n" + "=" * 60)
print("RESUMEN DE CAMBIOS")
print("=" * 60)
print(f"Registros iniciales:    {registros_iniciales:,}")
print(f"Registros finales:      {registros_finales:,}")
print(f"Registros eliminados:   {registros_eliminados:,}")
print(f"Porcentaje eliminado:   {porcentaje_eliminado:.2f}%")
print(f"Porcentaje conservado:  {100 - porcentaje_eliminado:.2f}%")

## Municipios

In [ ]:
# Validar y reclasificar municipios según código DANE
# Solo los municipios de Casanare (código inicia con 85) mantienen su nombre
# Los demás se agrupan en "Otro Departamento"

df_ms_sie['municipio'] = df_ms_sie.apply(
    lambda row: row['municipio'] if str(row['ID_COD_municipio']).startswith('85') else 'Otro Departamento',
    axis=1
)

# Verificar los resultados
print("=== RESUMEN DE RECLASIFICACIÓN DE MUNICIPIOS ===")
print("\nConteo de registros por categoría:")
print(df_ms_sie['municipio'].value_counts())

# Mostrar algunos ejemplos de municipios reclasificados
print("\n=== EJEMPLOS DE REGISTROS RECLASIFICADOS ===")
otros_dept = df_ms_sie[df_ms_sie['municipio'] == 'Otro Departamento'][['numero_identificacion', 'municipio', 'ID_COD_municipio']].head(10)
print(otros_dept.to_string(index=False))

print(f"\nTotal de registros en 'Otro Departamento': {(df_ms_sie['municipio'] == 'Otro Departamento').sum()}")
print(f"Total de registros en municipios de Casanare: {(df_ms_sie['municipio'] != 'Otro Departamento').sum()}")

## Telefono

In [ ]:
# Limpiar las columnas de teléfonos dejando solo números
df_ms_sie['celular'] = df_ms_sie['celular'].str.replace(r'\D', '', regex=True)
df_ms_sie['telefono_1'] = df_ms_sie['telefono_1'].str.replace(r'\D', '', regex=True)
df_ms_sie['telefono_2'] = df_ms_sie['telefono_2'].str.replace(r'\D', '', regex=True)

In [ ]:
# --- 1. Definición de Herramientas ---

def obtener_estadisticas(df, columnas, etapa):
    """
    Genera un resumen de cuántos registros tienen datos y cuántos son nulos/vacíos.
    Aplica el principio DRY (Don't Repeat Yourself).
    """
    metricas = []
    for col in columnas:
        if col in df.columns:
            # Contamos nulos reales (NaN) y cadenas vacías como "Vacíos"
            total = len(df)
            nulos_reales = df[col].isna().sum()
            vacios_str = (df[col] == '').sum()
            total_vacios = nulos_reales + vacios_str
            con_datos = total - total_vacios
            
            metricas.append({
                'Columna': col,
                'Etapa': etapa,
                'Registros con Datos': con_datos,
                'Registros Vacíos': total_vacios,
                '% Datos': round((con_datos / total) * 100, 2)
            })
    return pd.DataFrame(metricas)

def validar_telefono_colombia(numero):
    """
    Valida reglas de negocio para celulares en Colombia.
    Retorna el número limpio o pd.NA si es inválido.
    """
    # Manejo de nulos inicial
    if pd.isna(numero) or numero == '':
        return pd.NA  # Retornamos pd.NA directamente para uniformidad
    
    # Limpieza básica: convertimos a string y quitamos espacios
    # OJO: Si hay puntos decimales (ej: 300.0) esto podría fallar, aseguramos int conversion si aplica
    try:
        if isinstance(numero, float) and numero.is_integer():
             numero_str = str(int(numero))
        else:
             numero_str = str(numero).strip()
    except:
        return pd.NA

    # Reglas de Negocio
    # 1. Longitud exacta de 10
    if len(numero_str) != 10:
        return pd.NA
    
    # 2. Inicia con 3
    if not numero_str.startswith('3'):
        return pd.NA
    
    # 3. No todos los dígitos iguales (ej: 3333333333)
    if len(set(numero_str)) == 1:
        return pd.NA
    
    # 4. No patrón repetido después del 3 (ej: 3111111111)
    if len(set(numero_str[1:])) == 1:
        return pd.NA
    
    return numero_str

# --- 2. Captura de Estado Inicial (ANTES) ---

cols_objetivo = ['celular', 'telefono_1', 'telefono_2']
# Filtramos solo las que existan en el DF para evitar errores
cols_existentes = [c for c in cols_objetivo if c in df_ms_sie.columns]

print("--- Iniciando proceso de limpieza ---")
stats_antes = obtener_estadisticas(df_ms_sie, cols_existentes, 'ANTES')

# --- 3. Ejecución de la Limpieza ---

for col in cols_existentes:
    # Aplicamos la validación
    df_ms_sie[col] = df_ms_sie[col].apply(validar_telefono_colombia)

# --- 4. Captura de Estado Final (DESPUÉS) ---

stats_despues = obtener_estadisticas(df_ms_sie, cols_existentes, 'DESPUÉS')

# --- 5. Reporte Comparativo ---

# Unimos los dataframes y calculamos la diferencia (pérdida de datos)
reporte = pd.merge(stats_antes, stats_despues, on='Columna', suffixes=('_Antes', '_Despues'))

# Calculamos cuántos datos se eliminaron por ser inválidos
reporte['Datos Eliminados'] = reporte['Registros con Datos_Antes'] - reporte['Registros con Datos_Despues']

# Reordenamos columnas para lectura fácil
cols_finales = [
    'Columna', 
    'Registros con Datos_Antes', 'Registros con Datos_Despues', 
    'Datos Eliminados',
    'Registros Vacíos_Antes', 'Registros Vacíos_Despues'
]

print("\n=== REPORTE DE CALIDAD DE DATOS (TELÉFONOS) ===")
# Usamos to_string para que pandas no oculte filas/columnas si es muy grande
print(reporte[cols_finales].to_string(index=False))

print("\nProceso finalizado.")

In [ ]:
# Crear tabla con conteos (enteros)
campos = ['direccion', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']
datos_conteos = []

for campo in campos:
    total = len(df_ms_sie)
    con_datos = df_ms_sie[campo].notna().sum()
    vacios = total - con_datos
    datos_conteos.append({
        'Campo': campo,
        'Registros con Datos': con_datos,
        'Registros Vacíos': vacios
    })

df_conteos = pd.DataFrame(datos_conteos)

print("=== TABLA DE CONTEOS (ENTEROS) ===")
print(df_conteos.to_string(index=False))

# Crear tabla con porcentajes
datos_porcentajes = []

for campo in campos:
    total = len(df_ms_sie)
    con_datos = df_ms_sie[campo].notna().sum()
    vacios = total - con_datos
    pct_con_datos = round((con_datos / total) * 100, 2)
    pct_vacios = round((vacios / total) * 100, 2)
    datos_porcentajes.append({
        'Campo': campo,
        'Registros con Datos (%)': pct_con_datos,
        'Registros Vacíos (%)': pct_vacios
    })

df_porcentajes = pd.DataFrame(datos_porcentajes)

print("\n=== TABLA DE PORCENTAJES ===")
print(df_porcentajes.to_string(index=False))

In [ ]:
import pandas as pd

def generar_top_telefonos_validos(df):
    """
    Cuenta frecuencias de teléfonos únicos por usuario, filtrando estrictamente
    números que no cumplan con la longitud de 10 dígitos.
    Optimizado vectorialmente (sin bucles for).
    """
    
    # 1. 'Derretimos' (melt) el DataFrame: pasamos de 3 columnas a 1 sola columna larga
    # Esto pone todos los teléfonos en una sola columna llamada 'numero' manteniendo el índice original
    df_long = df.melt(
        value_vars=['celular', 'telefono_1', 'telefono_2'], 
        value_name='numero'
    ).dropna(subset=['numero']) # Eliminamos nulos iniciales

    # 2. Limpieza de seguridad y Conversión
    # Convertimos a string, quitamos espacios y decimales raros (.0) si existen
    df_long['numero'] = df_long['numero'].astype(str).str.replace(r'\.0$', '', regex=True).str.strip()

    # 3. FILTRO ESTRICTO (Aquí solucionamos lo del '300')
    # Solo dejamos pasar lo que tenga exactamente 10 caracteres
    df_long = df_long[df_long['numero'].str.len() == 10]

    # 4. Eliminamos duplicados POR USUARIO (por índice)
    # Si el usuario 1 tiene el número X en 'celular' y en 'telefono_1', 
    # aquí nos aseguramos de que solo cuente 1 vez.
    # drop_duplicates() sin argumentos mira todas las columnas (índice original + numero)
    df_unicos_por_usuario = df_long.loc[~df_long.index.duplicated(keep='first') | ~df_long.duplicated(subset=['numero'], keep=False)]
    # Corrección lógica más simple para asegurar unicidad (Index + Valor):
    # Reseteamos el index para que el ID del usuario sea una columna, y borramos duplicados de par (ID, Numero)
    df_unicos = df_long.reset_index().drop_duplicates(subset=['index', 'numero'])

    # 5. Conteo de frecuencias Globales
    conteo = df_unicos['numero'].value_counts().reset_index()
    conteo.columns = ['Número', 'Cantidad de Usuarios']
    
    return conteo

# --- Ejecución ---

print("--- Analizando números de teléfono más repetidos ---")
print("(Validando estrictamente longitud de 10 dígitos y unicidad por usuario)")

# Obtenemos el conteo global
df_frecuencias = generar_top_telefonos_validos(df_ms_sie)

# Ordenamos
df_frecuencias = df_frecuencias.sort_values('Cantidad de Usuarios', ascending=False)

# --- CORRECCIÓN DEL WARNING ---
# Usamos .copy() para crear un objeto independiente, no una vista
top_10 = df_frecuencias.head(10).copy()

print("\n=== TOP 10 NÚMEROS MÁS REPETIDOS ===")
print(top_10.to_string(index=False))

# Calcular porcentaje
# Ahora podemos modificar 'top_10' tranquilamente porque es una copia independiente
total_registros = len(df_ms_sie)
top_10['% del Total de Usuarios'] = (top_10['Cantidad de Usuarios'] / total_registros * 100).round(2)

print("\n=== TOP 10 CON PORCENTAJES ===")
print(top_10.to_string(index=False))

## Corrreos

In [ ]:
import pandas as pd
import re

# --- 1. Definición de la Lógica de Limpieza ---

def validar_correo_electronico(correo):
    """
    Valida estructura y limpia correos.
    Protege contra falsos positivos en la lista negra.
    Versión mejorada con detección de correos institucionales y basura.
    """
    # Manejo de nulos inicial
    if pd.isna(correo) or str(correo).strip() == '':
        return pd.NA
    
    # Normalización: minúsculas y sin espacios
    correo_str = str(correo).strip().lower()
    
    # Lista de correos basura conocidos (Blacklist) - AMPLIADA
    correos_basura_exactos = {
        # Variantes de "no tiene"
        'notiene@notiene.com', 'notiene@gmail.com', 'notienecorreo@gmail.com', 
        'notien@gmail.com', 'notiene@hotmail.com',
        
        # Variantes de "sin correo"
        'sincorreo@sincorreo.com', 'sincorreo@gmail.com',
        
        # Variantes de "no correo"
        'nocorreo@gmail.com', 'nocorreo@gmial.com', 'nocorreo@hotmail.com',
        
        # Variantes de "actualizar"
        'actualizar@actualizar.com', 'actualizar@gmail.com', 
        'actualizar.actualizar@gmail.com',
        
        # Correo institucional genérico
        'referenciacapresoca@capresoca-casanare.gov.co',
        
        # Otros comunes
        'no tiene', 'sin correo', 'no@correo.com', 'sin@correo.com'
    }
    
    # Prefijos sospechosos (más granular)
    prefijos_basura = [
        'notiene', 'nocorreo', 'sincorreo', 'no@', 'sin@', 
        'actualizar', 'notengo', 'referencia'
    ]
    
    # Palabras clave en el usuario (parte antes del @)
    palabras_basura_usuario = [
        'notiene', 'nocorreo', 'sincorreo', 'actualizar', 
        'notengo', 'referencia', 'sinmail', 'nocuenta'
    ]
    
    # 1. Chequeo de lista negra EXACTA
    if correo_str in correos_basura_exactos:
        return pd.NA
    
    # 2. Extraer usuario y dominio para análisis más profundo
    try:
        usuario, dominio = correo_str.split('@')
    except ValueError:
        return pd.NA  # No tiene @ o tiene más de uno
    
    # 3. Chequeo de palabras basura en el USUARIO
    for palabra in palabras_basura_usuario:
        if palabra in usuario:
            return pd.NA
    
    # 4. Chequeo de PREFIJOS (más seguro que 'in')
    for prefijo in prefijos_basura:
        if correo_str.startswith(prefijo):
            return pd.NA
    
    # 5. Validaciones de estructura
    # No debe tener comas (error de digitación común)
    if ',' in correo_str:
        return pd.NA
    
    # Detectar dominios mal escritos comunes (typos)
    dominios_typo = {
        'gmial.com': 'gmail.com',  # typo común
        'gmai.com': 'gmail.com',
        'hotmial.com': 'hotmail.com'
    }
    
    # Si el dominio es un typo conocido, lo marcamos como inválido
    # (porque probablemente el correo no funciona)
    if dominio in dominios_typo:
        return pd.NA
    
    # 6. Patrón regex estándar (RFC 5322 simplificado)
    patron_correo = r'^[a-zA-Z0-9._%-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
    
    if not re.match(patron_correo, correo_str):
        return pd.NA
    
    # 7. Verificación extra de dominio
    if '.' not in dominio:
        return pd.NA
    
    # 8. Usuario muy corto (menos de 3 caracteres es sospechoso)
    if len(usuario) < 3:
        return pd.NA
    
    return correo_str

# --- 2. Captura de Estado Inicial (ANTES) ---

print("=" * 70)
print("LIMPIEZA AVANZADA DE CORREOS ELECTRÓNICOS")
print("=" * 70)
print("\n--- Iniciando proceso de limpieza ---")

# Estadísticas ANTES
stats_correo_antes = obtener_estadisticas(df_ms_sie, ['correo_electronico'], 'ANTES')

# Guardar una muestra de correos sospechosos ANTES de limpiar
print("\n--- MUESTRA DE CORREOS SOSPECHOSOS (ANTES DE LIMPIEZA) ---")
correos_actuales = df_ms_sie['correo_electronico'].dropna()
if not correos_actuales.empty:
    # Buscar correos que contengan palabras sospechosas
    patron_sospechoso = r'(notiene|nocorreo|sincorreo|actualizar|referencia)'
    correos_sospechosos = correos_actuales[
        correos_actuales.str.lower().str.contains(patron_sospechoso, na=False)
    ].value_counts().head(20)
    
    if not correos_sospechosos.empty:
        print(f"\nSe encontraron {len(correos_sospechosos)} correos sospechosos únicos:")
        print(correos_sospechosos.to_string())
    else:
        print("No se encontraron correos con patrones sospechosos.")

# --- 3. Ejecución de la Limpieza ---

print("\n--- Aplicando validaciones y limpieza ---")
df_ms_sie['correo_electronico'] = df_ms_sie['correo_electronico'].apply(validar_correo_electronico)

# --- 4. Captura de Estado Final (DESPUÉS) ---

stats_correo_despues = obtener_estadisticas(df_ms_sie, ['correo_electronico'], 'DESPUÉS')

# --- 5. Reporte Comparativo Detallado ---

reporte_correo = pd.merge(
    stats_correo_antes, 
    stats_correo_despues, 
    on='Columna', 
    suffixes=('_Antes', '_Despues')
)

reporte_correo['Datos Eliminados'] = (
    reporte_correo['Registros con Datos_Antes'] - 
    reporte_correo['Registros con Datos_Despues']
)

# Calcular porcentajes
total_registros = len(df_ms_sie)
reporte_correo['% Eliminado'] = (
    reporte_correo['Datos Eliminados'] / 
    reporte_correo['Registros con Datos_Antes'] * 100
).round(2)

# Columnas finales
cols_finales_correo = [
    'Columna', 
    'Registros con Datos_Antes', 
    'Registros con Datos_Despues', 
    'Datos Eliminados',
    '% Eliminado',
    'Registros Vacíos_Antes', 
    'Registros Vacíos_Despues'
]

print("\n" + "=" * 70)
print("REPORTE DE CALIDAD DE DATOS (CORREO ELECTRÓNICO)")
print("=" * 70)
print(reporte_correo[cols_finales_correo].to_string(index=False))

# --- 6. Desglose de Tipos de Correos Eliminados ---

print("\n" + "=" * 70)
print("RESUMEN DE NORMALIZACIÓN")
print("=" * 70)

datos_eliminados = reporte_correo['Datos Eliminados'].iloc[0]
pct_eliminado = reporte_correo['% Eliminado'].iloc[0]

print(f"\nCorreos eliminados/convertidos a vacío: {int(datos_eliminados):,}")
print(f"Porcentaje de correos basura eliminados: {pct_eliminado:.2f}%")

# Categorías de correos eliminados (estimación basada en patrones)
print("\n--- CATEGORÍAS DE CORREOS ELIMINADOS ---")
categorias = {
    'Variantes de "no tiene/correo"': ['notiene', 'nocorreo', 'notien'],
    'Variantes de "sin correo"': ['sincorreo'],
    'Variantes de "actualizar"': ['actualizar'],
    'Correos institucionales genéricos': ['referencia', 'capresoca'],
    'Dominios con typos': ['gmial', 'hotmial'],
    'Formato inválido': ['sin @', 'sin dominio', 'muy corto']
}

for categoria, patrones in categorias.items():
    print(f"  - {categoria}: Incluye patrones como {', '.join(patrones[:2])}")

print("\n" + "=" * 70)
print("Proceso finalizado exitosamente.")
print("=" * 70)

In [ ]:
import pandas as pd

def generar_top_correos_validos(df):
    """
    Cuenta frecuencias de correos electrónicos únicos por usuario.
    Optimizado vectorialmente (sin bucles for).
    """
    
    # 1. Extraemos solo la columna 'correo_electronico'
    df_correos = df[['correo_electronico']].copy()
    
    # 2. Eliminamos nulos y valores vacíos
    df_correos = df_correos.dropna(subset=['correo_electronico'])
    
    # 3. Normalización: convertimos a minúsculas y quitamos espacios extra
    df_correos['correo_limpio'] = (
        df_correos['correo_electronico']
        .str.lower()
        .str.strip()
    )
    
    # 4. Filtramos correos vacíos después de la normalización
    df_correos = df_correos[df_correos['correo_limpio'] != '']
    
    # 5. Conteo de frecuencias
    conteo = df_correos['correo_limpio'].value_counts().reset_index()
    conteo.columns = ['Correo Electrónico', 'Cantidad de Usuarios']
    
    return conteo

# --- Ejecución ---

print("=" * 70)
print("ANÁLISIS DE CORREOS ELECTRÓNICOS MÁS REPETIDOS")
print("=" * 70)

# Obtenemos el conteo global
df_frecuencias_correo = generar_top_correos_validos(df_ms_sie)

# --- CORRECCIÓN DEL WARNING ---
# Usamos .copy() para crear un objeto independiente, no una vista
top_10_correos = df_frecuencias_correo.head(10).copy()

print("\n=== TOP 10 CORREOS MÁS REPETIDOS ===")
print(top_10_correos.to_string(index=False))

# Calcular porcentaje
total_registros = len(df_ms_sie)
top_10_correos['% del Total de Usuarios'] = (
    top_10_correos['Cantidad de Usuarios'] / total_registros * 100
).round(2)

print("\n=== TOP 10 CON PORCENTAJES ===")
print(top_10_correos.to_string(index=False))

# Estadísticas adicionales
print("\n" + "=" * 70)
print("ESTADÍSTICAS ADICIONALES")
print("=" * 70)
print(f"Total de registros en el dataset: {total_registros:,}")
print(f"Correos únicos encontrados: {len(df_frecuencias_correo):,}")
print(f"Registros con correo válido: {df_frecuencias_correo['Cantidad de Usuarios'].sum():,}")
print(f"Registros sin correo: {total_registros - df_frecuencias_correo['Cantidad de Usuarios'].sum():,}")

# Análisis de dominios más comunes
print("\n" + "=" * 70)
print("TOP 10 DOMINIOS MÁS USADOS")
print("=" * 70)

# Extraer dominios de los correos
df_frecuencias_correo['Dominio'] = df_frecuencias_correo['Correo Electrónico'].str.split('@').str[1]
dominios_top = df_frecuencias_correo.groupby('Dominio')['Cantidad de Usuarios'].sum().sort_values(ascending=False).head(10)

for i, (dominio, count) in enumerate(dominios_top.items(), 1):
    pct = (count / total_registros) * 100
    print(f"{i:2d}. {dominio:30s} | {count:6,} usuarios ({pct:5.2f}%)")

## Direcciones

In [ ]:
import pandas as pd

def limpiar_direcciones_ruido(df):
    """
    Limpia direcciones que son ruido o información no útil.
    Convierte a NaN las direcciones que no aportan valor.
    """
    # Lista de valores que se consideran "ruido" o no informativos
    valores_ruido = {
        'ACTUALIZAR', 'CENTRO', 'NO TIENE', 'N', 'NO', 'SIN DIRECCION',
        'SIN DIRECCIÓN', 'NINGUNA', 'NO REGISTRA', 'NO APLICA', 'N/A',
        'NA', 'DESCONOCIDO', 'DESCONOCIDA', 'NO SABE', '.', '-', 'X'
    }
    
    # Función para validar cada dirección
    def validar_direccion(direccion):
        # Si es nulo, mantener nulo
        if pd.isna(direccion):
            return pd.NA
        
        # Normalizar: mayúsculas y sin espacios extra
        dir_limpia = str(direccion).strip().upper()
        
        # Si está vacío después de limpiar
        if dir_limpia == '':
            return pd.NA
        
        # Si está en la lista de ruido
        if dir_limpia in valores_ruido:
            return pd.NA
        
        # Si es muy corto (menos de 3 caracteres) probablemente no es una dirección real
        if len(dir_limpia) < 3:
            return pd.NA
        
        # Si solo contiene puntos, guiones o espacios
        if all(c in '.- ' for c in dir_limpia):
            return pd.NA
        
        # Si pasó todas las validaciones, mantener la dirección normalizada
        return dir_limpia
    
    return df['direccion'].apply(validar_direccion)

# --- ANÁLISIS ANTES DE LA LIMPIEZA ---

print("=" * 70)
print("LIMPIEZA DE DIRECCIONES CON RUIDO")
print("=" * 70)

# Capturar estado inicial
registros_totales = len(df_ms_sie)
direcciones_antes = df_ms_sie['direccion'].notna().sum()
direcciones_vacias_antes = df_ms_sie['direccion'].isna().sum()

print("\n--- ESTADO ANTES DE LA LIMPIEZA ---")
print(f"Total de registros: {registros_totales:,}")
print(f"Direcciones con datos: {direcciones_antes:,} ({direcciones_antes/registros_totales*100:.2f}%)")
print(f"Direcciones vacías: {direcciones_vacias_antes:,} ({direcciones_vacias_antes/registros_totales*100:.2f}%)")

# Mostrar ejemplos de direcciones sospechosas (antes de limpiar)
print("\n--- MUESTRA DE DIRECCIONES SOSPECHOSAS (ANTES) ---")
direcciones_cortas = df_ms_sie[df_ms_sie['direccion'].str.len() < 10]['direccion'].value_counts().head(15)
if not direcciones_cortas.empty:
    print(direcciones_cortas.to_string())
else:
    print("No se encontraron direcciones cortas.")

# --- APLICAR LA LIMPIEZA ---

df_ms_sie['direccion'] = limpiar_direcciones_ruido(df_ms_sie)

# --- ANÁLISIS DESPUÉS DE LA LIMPIEZA ---

direcciones_despues = df_ms_sie['direccion'].notna().sum()
direcciones_vacias_despues = df_ms_sie['direccion'].isna().sum()
direcciones_normalizadas = direcciones_antes - direcciones_despues

print("\n" + "=" * 70)
print("--- ESTADO DESPUÉS DE LA LIMPIEZA ---")
print("=" * 70)
print(f"Total de registros: {registros_totales:,}")
print(f"Direcciones con datos: {direcciones_despues:,} ({direcciones_despues/registros_totales*100:.2f}%)")
print(f"Direcciones vacías: {direcciones_vacias_despues:,} ({direcciones_vacias_despues/registros_totales*100:.2f}%)")

print("\n" + "=" * 70)
print("--- RESUMEN DE NORMALIZACIÓN ---")
print("=" * 70)
print(f"Direcciones antes de limpiar:    {direcciones_antes:,}")
print(f"Direcciones después de limpiar:  {direcciones_despues:,}")
print(f"Direcciones convertidas a vacío: {direcciones_normalizadas:,}")
print(f"Porcentaje de ruido eliminado:   {direcciones_normalizadas/direcciones_antes*100:.2f}%" if direcciones_antes > 0 else "N/A")

# Mostrar las 20 direcciones más comunes después de la limpieza
print("\n--- TOP 20 DIRECCIONES MÁS COMUNES (DESPUÉS DE LIMPIEZA) ---")
top_direcciones_limpias = df_ms_sie['direccion'].value_counts().head(20)
if not top_direcciones_limpias.empty:
    for i, (direccion, count) in enumerate(top_direcciones_limpias.items(), 1):
        pct = (count / registros_totales) * 100
        print(f"{i:2d}. {direccion[:50]:50s} | {count:6,} ({pct:5.2f}%)")
else:
    print("No hay direcciones válidas después de la limpieza.")

In [ ]:
import pandas as pd

def generar_top_direcciones_validas(df):
    """
    Cuenta frecuencias de direcciones únicas.
    Optimizado vectorialmente (sin bucles for).
    """
    
    # 1. Extraemos solo la columna 'direccion'
    df_direcciones = df[['direccion']].copy()
    
    # 2. Eliminamos nulos y valores vacíos
    df_direcciones = df_direcciones.dropna(subset=['direccion'])
    df_direcciones = df_direcciones[df_direcciones['direccion'].str.strip() != '']
    
    # 3. Normalización básica (opcional pero recomendado)
    # Convertimos a mayúsculas y quitamos espacios extra
    df_direcciones['direccion_limpia'] = (
        df_direcciones['direccion']
        .str.upper()
        .str.strip()
        .str.replace(r'\s+', ' ', regex=True)  # Múltiples espacios a uno solo
    )
    
    # 4. Conteo de frecuencias
    conteo = df_direcciones['direccion_limpia'].value_counts().reset_index()
    conteo.columns = ['Dirección', 'Cantidad de Usuarios']
    
    return conteo

# --- Ejecución ---

print("=" * 70)
print("ANÁLISIS DE DIRECCIONES MÁS REPETIDAS")
print("=" * 70)

# Obtenemos el conteo global
df_frecuencias_dir = generar_top_direcciones_validas(df_ms_sie)

# --- CORRECCIÓN DEL WARNING ---
# Usamos .copy() para crear un objeto independiente, no una vista
top_10_direcciones = df_frecuencias_dir.head(10).copy()

print("\n=== TOP 10 DIRECCIONES MÁS REPETIDAS ===")
print(top_10_direcciones.to_string(index=False))

# Calcular porcentaje
total_registros = len(df_ms_sie)
top_10_direcciones['% del Total de Usuarios'] = (
    top_10_direcciones['Cantidad de Usuarios'] / total_registros * 100
).round(2)

print("\n=== TOP 10 CON PORCENTAJES ===")
print(top_10_direcciones.to_string(index=False))

# Estadísticas adicionales
print("\n" + "=" * 70)
print("ESTADÍSTICAS ADICIONALES")
print("=" * 70)
print(f"Total de registros en el dataset: {total_registros:,}")
print(f"Direcciones únicas encontradas: {len(df_frecuencias_dir):,}")
print(f"Registros con dirección válida: {df_frecuencias_dir['Cantidad de Usuarios'].sum():,}")
print(f"Registros sin dirección: {total_registros - df_frecuencias_dir['Cantidad de Usuarios'].sum():,}")

# Graficas

## General

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# --- Cálculo de datos ---
campos = ['direccion', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']

resumen = df_ms_sie[campos].notna().agg(['sum', 'count']).T
resumen.columns = ['Con Datos', 'Total']
resumen['Vacíos'] = resumen['Total'] - resumen['Con Datos']
resumen['% Con Datos'] = (resumen['Con Datos'] / resumen['Total']) * 100
resumen['% Vacíos'] = (resumen['Vacíos'] / resumen['Total']) * 100
resumen = resumen.sort_values('% Con Datos', ascending=False)

print("--- Resumen de Calidad ---")
print(resumen)

# --- Visualización Mejorada ---

def graficar_completitud(df_resumen):
    """
    Genera un gráfico de barras apiladas mostrando la completitud de los datos
    con valores enteros y porcentajes.
    """
    fig, ax = plt.subplots(figsize=(12, 7))
    
    # Definimos los ejes
    campos = df_resumen.index
    con_datos = df_resumen['% Con Datos']
    vacios = df_resumen['% Vacíos']
    
    # Valores enteros
    con_datos_int = df_resumen['Con Datos']
    vacios_int = df_resumen['Vacíos']
    
    # Colores semánticos
    color_datos = '#2ecc71'  # Verde esmeralda
    color_vacio = '#95a5a6'  # Gris concreto
    
    # Crear las barras
    bar1 = ax.bar(campos, con_datos, label='Con Datos', color=color_datos, edgecolor='white', linewidth=2)
    bar2 = ax.bar(campos, vacios, bottom=con_datos, label='Vacíos', color=color_vacio, edgecolor='white', linewidth=2)
    
    # Añadir etiquetas con valores enteros Y porcentajes
    for i, (rect_datos, rect_vacios) in enumerate(zip(bar1, bar2)):
        # Para la sección "Con Datos" (verde)
        height_datos = rect_datos.get_height()
        if height_datos > 5:
            valor_int = int(con_datos_int.iloc[i])
            pct = height_datos
            label_text = f'{valor_int:,}\n({pct:.1f}%)'
            ax.text(
                rect_datos.get_x() + rect_datos.get_width()/2,
                rect_datos.get_y() + height_datos/2,
                label_text,
                ha='center', va='center',
                color='white', fontweight='bold',
                fontsize=10
            )
        
        # Para la sección "Vacíos" (gris)
        height_vacios = rect_vacios.get_height()
        if height_vacios > 5:
            valor_int = int(vacios_int.iloc[i])
            pct = height_vacios
            label_text = f'{valor_int:,}\n({pct:.1f}%)'
            ax.text(
                rect_vacios.get_x() + rect_vacios.get_width()/2,
                rect_vacios.get_y() + height_vacios/2,
                label_text,
                ha='center', va='center',
                color='white', fontweight='bold',
                fontsize=10
            )

    # Personalización del gráfico
    ax.set_ylabel('Porcentaje de Completitud (%)', fontsize=11, fontweight='bold')
    ax.set_xlabel('Campos de Contacto', fontsize=11, fontweight='bold')
    ax.set_title('Calidad de Datos: Disponibilidad de Información por Campo', 
                 pad=20, fontweight='bold', fontsize=14)
    
    # Mejorar la leyenda
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.08), 
              ncol=2, frameon=True, shadow=True, fontsize=10)
    
    # Eliminar bordes innecesarios
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['left'].set_color('#CCCCCC')
    ax.spines['bottom'].set_color('#CCCCCC')
    
    # Cuadrícula suave
    ax.grid(axis='y', linestyle='--', alpha=0.3, color='gray')
    ax.set_axisbelow(True)
    
    # Ajustar límites del eje Y
    ax.set_ylim(0, 105)
    
    # Rotar etiquetas del eje X si es necesario
    plt.xticks(rotation=15, ha='right')
    
    plt.tight_layout()
    
    # Guardar o mostrar
    # plt.savefig('calidad_datos_resumen_mejorado.png', dpi=300, bbox_inches='tight')
    print("\nGráfico generado exitosamente.")
    plt.show()

# Ejecutar la función
graficar_completitud(resumen)

## Top 10 telefonos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# --- VISUALIZACIÓN DE RESULTADOS ---

# Validación de seguridad: Solo graficamos si hay datos
if not top_10.empty:
    
    # Configuración del estilo para reporte profesional
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(12, 6))

    # Crear el gráfico de barras horizontales
    # Usamos viridis_r para que el #1 tenga el color más intenso
    ax = sns.barplot(
        data=top_10,
        x='Cantidad de Usuarios',
        y='Número',
        palette='viridis_r'
    )

    # Títulos y Etiquetas
    plt.title(f'Top 10 Números de Teléfono Más Repetidos\n(Total Registros Analizados: {total_registros})', 
              fontsize=14, fontweight='bold', pad=20)
    plt.xlabel('Cantidad de Usuarios Únicos', fontsize=11)
    plt.ylabel('Número Telefónico', fontsize=11)

    # Añadir las anotaciones (Count y Porcentaje) al final de cada barra
    for i, p in enumerate(ax.patches):
        # Obtener el ancho de la barra (que es el valor de x)
        width = p.get_width()
        
        # Recuperar el porcentaje correspondiente desde el DataFrame
        pct = top_10.iloc[i]['% del Total de Usuarios']
        
        # Formatear el texto: " 150 (5.2%)"
        etiqueta = f' {int(width)} ({pct}%)'
        
        # Colocar el texto un poco a la derecha del final de la barra
        ax.text(width, p.get_y() + p.get_height() / 2, etiqueta, 
                ha='left', va='center', fontweight='bold', color='#333333')

    # Ajustes finales de limpieza visual
    sns.despine(left=True, bottom=True) # Quitar bordes innecesarios
    plt.tight_layout()

    # Guardar y Mostrar
    #plt.savefig('top_10_telefonos_repetidos.png', dpi=300)
    print("\nGráfico generado exitosamente: 'top_10_telefonos_repetidos.png'")
    plt.show()

else:
    print("\n[!] No hay datos para graficar. El DataFrame 'top_10' está vacío.")

## % vacios por muniicpio

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# --- PREPARACIÓN DE DATOS PARA MAPA DE CALOR ---

campos_analizar = ['direccion', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']

# Crear DataFrame con CONTEOS de vacíos por municipio
df_conteos = df_ms_sie.groupby('municipio').agg({
    **{campo: lambda x: x.isna().sum() for campo in campos_analizar},
    'municipio': 'count'
}).rename(columns={'municipio': 'Total Registros'})

# Crear DataFrame con PORCENTAJES de vacíos por municipio
df_porcentajes = df_ms_sie.groupby('municipio').agg({
    **{campo: lambda x: x.isna().mean() * 100 for campo in campos_analizar},
}).round(1)

# Ordenar por total de registros
df_conteos = df_conteos.sort_values('Total Registros', ascending=False)
df_porcentajes = df_porcentajes.loc[df_conteos.index]

# Calcular totales por columna y fila
totales_por_campo = df_conteos[campos_analizar].sum()
pct_por_campo = (totales_por_campo / df_conteos['Total Registros'].sum() * 100).round(1)
df_conteos['Total Vacíos'] = df_conteos[campos_analizar].sum(axis=1)
df_conteos['Promedio %'] = df_porcentajes[campos_analizar].mean(axis=1).round(1)

# Preparar matrices
matriz_calor = df_porcentajes
matriz_conteos = df_conteos[campos_analizar]

# --- CREAR ANOTACIONES PERSONALIZADAS (PORCENTAJE + CONTEO) ---

# Crear matriz de anotaciones con formato: "66.5%\n(36,123)"
anotaciones = np.empty(matriz_calor.shape, dtype=object)

for i in range(matriz_calor.shape[0]):
    for j in range(matriz_calor.shape[1]):
        pct = matriz_calor.iloc[i, j]
        count = int(matriz_conteos.iloc[i, j])
        anotaciones[i, j] = f'{pct:.1f}%\n({count:,})'

# --- VISUALIZACIÓN ---

altura = max(12, len(matriz_calor) * 0.5)
fig, ax = plt.subplots(figsize=(18, altura))

# Crear el mapa de calor con anotaciones personalizadas
sns.heatmap(
    matriz_calor,
    annot=anotaciones,
    fmt='',  # Importante: formato vacío porque usamos strings personalizados
    cmap='RdYlGn_r',
    cbar_kws={
        'label': '% de Registros Vacíos',
        'shrink': 0.8,
        'aspect': 30
    },
    linewidths=1.5,
    linecolor='white',
    vmin=0,
    vmax=100,
    annot_kws={'fontsize': 7, 'fontweight': 'bold'},
    ax=ax
)

# --- COLUMNA DE TOTALES POR FILA ---
for i, (idx, row) in enumerate(df_conteos.iterrows()):
    total_registros = int(row['Total Registros'])
    total_vacios = int(row['Total Vacíos'])
    promedio_pct = row['Promedio %']
    
    texto = f'{total_registros:,}\n({total_vacios:,})\n{promedio_pct:.1f}%'
    
    ax.text(
        len(campos_analizar) + 0.5, i + 0.5, texto,
        ha='center', va='center', fontsize=7, fontweight='bold',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='lightgray', alpha=0.5)
    )

# Encabezado columna totales
ax.text(
    len(campos_analizar) + 0.5, -0.5,
    'Total Registros\n(Total Vacíos)\nPromedio %',
    ha='center', va='center', fontsize=8, fontweight='bold',
    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightblue', alpha=0.7)
)

# --- FILA DE TOTALES POR COLUMNA ---
for j, campo in enumerate(campos_analizar):
    total_vacios_campo = int(totales_por_campo[campo])
    pct_campo = pct_por_campo[campo]
    
    texto = f'{total_vacios_campo:,}\n({pct_campo:.1f}%)'
    
    ax.text(
        j + 0.5, len(matriz_calor) + 0.5, texto,
        ha='center', va='center', fontsize=8, fontweight='bold',
        bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.5)
    )

# Etiqueta fila totales
ax.text(
    -0.5, len(matriz_calor) + 0.5,
    'TOTAL POR CAMPO\n(Vacíos y %)',
    ha='right', va='center', fontsize=8, fontweight='bold',
    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightyellow', alpha=0.7)
)

# Celda resumen general
total_general = df_conteos['Total Registros'].sum()
total_vacios_general = df_conteos['Total Vacíos'].sum()
pct_general = (total_vacios_general / (total_general * len(campos_analizar)) * 100).round(1)

ax.text(
    len(campos_analizar) + 0.5, len(matriz_calor) + 0.5,
    f'TOTAL\n{total_vacios_general:,}\n({pct_general:.1f}%)',
    ha='center', va='center', fontsize=8, fontweight='bold',
    bbox=dict(boxstyle='round,pad=0.3', facecolor='lightcoral', alpha=0.5)
)

# Títulos y etiquetas
plt.title(
    'Mapa de Calor: Datos Vacíos por Municipio (Porcentaje y Cantidad)\n' + 
    f'Total de Municipios: {len(matriz_calor)} | Total de Registros: {total_general:,}',
    fontsize=14, fontweight='bold', pad=20
)
plt.xlabel('Campos de Contacto', fontsize=11, fontweight='bold')
plt.ylabel('Municipio', fontsize=11, fontweight='bold')

plt.xticks(rotation=45, ha='right', fontsize=10)
plt.yticks(rotation=0, fontsize=9)

ax.set_xlim(0, len(campos_analizar) + 1)
ax.set_ylim(0, len(matriz_calor) + 1)

plt.tight_layout()

print("=" * 80)
print("MAPA DE CALOR GENERADO CON PORCENTAJES Y CONTEOS")
print("=" * 80)
print("\nEjemplo de lectura:")
print("Celda 'Yopal - correo_electronico': '66.5%\\n(36,123)'")
print("  → Significa: 66.5% de registros de Yopal no tienen correo")
print("  → Equivale a: 36,123 registros sin correo electrónico")
print("=" * 80)

plt.show()

## % por Regimen

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# --- 1. Preparación de Datos (Clean Code) ---
campos_analizar = ['direccion', 'celular', 'telefono_1', 'telefono_2', 'correo_electronico']

# Calculamos PORCENTAJES de vacíos
df_resumen_pct = df_ms_sie.groupby('regimen')[campos_analizar].apply(
    lambda x: x.isna().mean() * 100
).reset_index()

# Calculamos CONTEOS (enteros) de vacíos
df_resumen_count = df_ms_sie.groupby('regimen')[campos_analizar].apply(
    lambda x: x.isna().sum()
).reset_index()

# Obtener totales por régimen para referencia
totales_por_regimen = df_ms_sie.groupby('regimen').size().to_dict()

# Transformamos para gráfico
df_melted_pct = df_resumen_pct.melt(
    id_vars='regimen', 
    value_vars=campos_analizar, 
    var_name='Campo', 
    value_name='Porcentaje_Vacios'
)

df_melted_count = df_resumen_count.melt(
    id_vars='regimen', 
    value_vars=campos_analizar, 
    var_name='Campo', 
    value_name='Cantidad_Vacios'
)

# Unimos ambos DataFrames
df_melted = df_melted_pct.merge(df_melted_count, on=['regimen', 'Campo'])

# --- 2. Visualización Profesional ---

plt.figure(figsize=(14, 7))
sns.set_theme(style="whitegrid")

# Creamos el Gráfico de Barras Agrupadas
ax = sns.barplot(
    data=df_melted,
    x='Campo',
    y='Porcentaje_Vacios',
    hue='regimen',
    palette='viridis',
    edgecolor='white',
    linewidth=1.5
)

# --- 3. Personalización y Etiquetas ---

plt.title('Calidad de Datos: Comparativa de Vacíos por Régimen\n(Porcentaje y Cantidad Absoluta)', 
          fontsize=16, fontweight='bold', pad=20)
plt.ylabel('% de Registros Vacíos (Menos es Mejor)', fontsize=12)
plt.xlabel('Campos de Contacto', fontsize=12)

# Ajuste del eje Y
plt.ylim(0, 115)

# Leyenda
plt.legend(title='Régimen', title_fontsize='11', fontsize='10', 
           loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=2, frameon=False)

# --- 4. Etiquetas de Datos PERSONALIZADAS (Porcentaje + Conteo) ---

# Iteramos sobre cada contenedor (cada régimen)
for container in ax.containers:
    # Obtenemos las etiquetas personalizadas
    labels = []
    
    for bar in container:
        # Obtenemos la altura de la barra (porcentaje)
        height = bar.get_height()
        
        # Obtenemos la posición x de la barra
        x_pos = bar.get_x() + bar.get_width() / 2
        
        # Determinamos el campo y régimen correspondiente
        # (Seaborn organiza las barras en orden)
        bar_index = container.index(bar)
        
        # Buscamos el conteo correspondiente en df_melted
        # Filtramos por la posición relativa
        try:
            # Obtenemos el índice correcto del dataframe melted
            regimen_idx = ax.containers.index(container)
            campo_idx = bar_index
            
            # Calculamos el índice en df_melted
            df_idx = regimen_idx * len(campos_analizar) + campo_idx
            
            if df_idx < len(df_melted):
                cantidad = int(df_melted.iloc[df_idx]['Cantidad_Vacios'])
                label = f'{height:.1f}%\n({cantidad:,})'
            else:
                label = f'{height:.1f}%'
        except:
            label = f'{height:.1f}%'
        
        labels.append(label)
    
    # Aplicamos las etiquetas
    ax.bar_label(container, labels=labels, padding=3, fontsize=8, fontweight='bold')

# Limpieza final
sns.despine(left=True)
plt.tight_layout()

# --- 5. Reportes en Consola ---

print("=" * 80)
print("RESUMEN DE VACÍOS POR RÉGIMEN")
print("=" * 80)

# Mostrar tabla con porcentajes
print("\n--- PORCENTAJES DE VACÍOS (%) ---")
print(df_resumen_pct.to_string(index=False))

# Mostrar tabla con conteos
print("\n--- CANTIDAD DE REGISTROS VACÍOS (Enteros) ---")
print(df_resumen_count.to_string(index=False))

# Totales por régimen
print("\n--- TOTAL DE REGISTROS POR RÉGIMEN ---")
for regimen, total in totales_por_regimen.items():
    print(f"{regimen}: {total:,} registros")

# Análisis comparativo
print("\n" + "=" * 80)
print("ANÁLISIS COMPARATIVO")
print("=" * 80)

for campo in campos_analizar:
    print(f"\n{campo.upper()}:")
    for regimen in df_resumen_pct['regimen'].unique():
        pct = df_resumen_pct[df_resumen_pct['regimen'] == regimen][campo].values[0]
        count = df_resumen_count[df_resumen_count['regimen'] == regimen][campo].values[0]
        total = totales_por_regimen[regimen]
        
        print(f"  {regimen:20s} | {pct:5.1f}% vacío | {int(count):6,} registros | de {total:,} total")

print("\n" + "=" * 80)
print("Gráfico generado exitosamente")
print("=" * 80)

plt.show()

## top 10 dirección

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# --- VISUALIZACIÓN DE DIRECCIONES MÁS REPETIDAS ---

# Validación de seguridad: Solo graficamos si hay datos
if not top_10_direcciones.empty:
    
    # Configuración del estilo para reporte profesional
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(14, 8))  # Más alto para direcciones largas

    # Crear el gráfico de barras horizontales
    ax = sns.barplot(
        data=top_10_direcciones,
        x='Cantidad de Usuarios',
        y='Dirección',
        palette='viridis_r',
        orient='h'
    )

    # Títulos y Etiquetas
    plt.title(f'Top 10 Direcciones Más Repetidas\n(Total Registros Analizados: {total_registros:,})', 
              fontsize=14, fontweight='bold', pad=20)
    plt.xlabel('Cantidad de Usuarios', fontsize=11)
    plt.ylabel('Dirección', fontsize=11)

    # Añadir las anotaciones (Count y Porcentaje) al final de cada barra
    for i, p in enumerate(ax.patches):
        width = p.get_width()
        pct = top_10_direcciones.iloc[i]['% del Total de Usuarios']
        
        # Formatear el texto: " 150 (5.2%)"
        etiqueta = f' {int(width):,} ({pct}%)'
        
        ax.text(width, p.get_y() + p.get_height() / 2, etiqueta, 
                ha='left', va='center', fontweight='bold', color='#333333', fontsize=10)

    # Ajustar el tamaño de las etiquetas del eje Y para direcciones largas
    ax.tick_params(axis='y', labelsize=9)
    
    # Ajustes finales de limpieza visual
    sns.despine(left=True, bottom=True)
    plt.tight_layout()

    # Guardar y Mostrar
    #plt.savefig('top_10_direcciones_repetidas.png', dpi=300, bbox_inches='tight')
    print("\nGráfico generado exitosamente: 'top_10_direcciones_repetidas.png'")
    plt.show()

else:
    print("\n[!] No hay datos para graficar. El DataFrame 'top_10_direcciones' está vacío.")

## Top 10 correos

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# --- VISUALIZACIÓN DE CORREOS MÁS REPETIDOS ---

# Validación de seguridad: Solo graficamos si hay datos
if not top_10_correos.empty:
    
    # Configuración del estilo para reporte profesional
    sns.set_theme(style="whitegrid")
    plt.figure(figsize=(14, 8))  # Más alto para correos largos

    # Crear el gráfico de barras horizontales
    ax = sns.barplot(
        data=top_10_correos,
        x='Cantidad de Usuarios',
        y='Correo Electrónico',
        palette='viridis_r',
        orient='h'
    )

    # Títulos y Etiquetas
    plt.title(f'Top 10 Correos Electrónicos Más Repetidos\n(Total Registros Analizados: {total_registros:,})', 
              fontsize=14, fontweight='bold', pad=20)
    plt.xlabel('Cantidad de Usuarios', fontsize=11)
    plt.ylabel('Correo Electrónico', fontsize=11)

    # Añadir las anotaciones (Count y Porcentaje) al final de cada barra
    for i, p in enumerate(ax.patches):
        width = p.get_width()
        pct = top_10_correos.iloc[i]['% del Total de Usuarios']
        
        # Formatear el texto: " 150 (5.2%)"
        etiqueta = f' {int(width):,} ({pct}%)'
        
        ax.text(width, p.get_y() + p.get_height() / 2, etiqueta, 
                ha='left', va='center', fontweight='bold', color='#333333', fontsize=10)

    # Ajustar el tamaño de las etiquetas del eje Y para correos largos
    ax.tick_params(axis='y', labelsize=9)
    
    # Ajustes finales de limpieza visual
    sns.despine(left=True, bottom=True)
    plt.tight_layout()

    # Guardar y Mostrar
    #plt.savefig('top_10_correos_repetidos.png', dpi=300, bbox_inches='tight')
    print("\nGráfico generado exitosamente: 'top_10_correos_repetidos.png'")
    plt.show()

else:
    print("\n[!] No hay datos para graficar. El DataFrame 'top_10_correos' está vacío.")

In [ ]:
print(df_ms_sie['estado'].value_counts())

In [ ]:
print(df_ms_sie.columns)

In [ ]:
from pathlib import Path
import pandas as pd

# Definir la ruta de salida
ruta_salida = Path(r"C:\Users\osmarrincon\Downloads")
nombre_archivo = "SIE_Aseguramiento_Original_y_Procesado.xlsx"
archivo_completo = ruta_salida / nombre_archivo

print("=" * 70)
print("EXPORTACIÓN DE DATOS A EXCEL")
print("=" * 70)

# Cargar el DataFrame original desde el CSV
print("\n[1/3] Cargando datos originales desde el CSV...")
df_original = pd.read_csv(r_ms_sie, sep=';', dtype=str, encoding='latin-1')
print(f"✓ Datos originales cargados: {len(df_original):,} registros")

# Exportar ambos DataFrames a un archivo Excel con múltiples hojas
print("\n[2/3] Exportando a Excel con dos hojas...")
with pd.ExcelWriter(archivo_completo, engine='openpyxl') as writer:
    # Hoja 1: Datos originales
    df_original.to_excel(writer, sheet_name='Datos Originales', index=False)
    
    # Hoja 2: Datos procesados (limpios)
    df_ms_sie.to_excel(writer, sheet_name='Datos Procesados', index=False)

print(f"✓ Archivo Excel creado exitosamente")

# Verificar que el archivo se creó
if archivo_completo.exists():
    tamaño_mb = archivo_completo.stat().st_size / (1024 * 1024)
    print("\n[3/3] Verificación completada")
    print(f"✓ Archivo guardado en: {archivo_completo}")
    print(f"✓ Tamaño del archivo: {tamaño_mb:.2f} MB")
    
    print("\n" + "=" * 70)
    print("RESUMEN DE CONTENIDO")
    print("=" * 70)
    print(f"\nHoja 1: 'Datos Originales'")
    print(f"  - Registros: {len(df_original):,}")
    print(f"  - Columnas: {len(df_original.columns)}")
    
    print(f"\nHoja 2: 'Datos Procesados'")
    print(f"  - Registros: {len(df_ms_sie):,}")
    print(f"  - Columnas: {len(df_ms_sie.columns)}")
    print(f"  - Registros eliminados: {len(df_original) - len(df_ms_sie):,}")
    print(f"  - Porcentaje conservado: {(len(df_ms_sie)/len(df_original))*100:.2f}%")
    
    print("\n" + "=" * 70)
    print("✓ EXPORTACIÓN COMPLETADA EXITOSAMENTE")
    print("=" * 70)
else:
    print("\n[!] ERROR: No se pudo crear el archivo")